# Wolfram Oracle Bet example

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***


In [Marlowe Playground](https://play.marlowe.iohk.io//), the contract looks like this in Blockly format.


In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Role "Alice")
            (Role "Alice")
            (Token "" "")
            (Constant 1000)
        )
        (When
            [Case
                (Deposit
                    (Role "Bob")
                    (Role "Bob")
                    (Token "" "")
                    (Constant 1000)
                )
                (When
                    [Case
                        (Choice
                            (ChoiceId
                                "WOLF_BTCUSD"
                                (Address "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j")
                            )
                            [Bound 0 100000000]
                        )
                        (If
                            (ValueLT
                                (ChoiceValue
                                    (ChoiceId
                                        "WOLF_BTCUSD"
                                        (Address "addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j")
                                    ))
                                (Constant 1000000)
                            )
                            (Pay
                                (Role "Alice")
                                (Party (Role "Bob"))
                                (Token "" "")
                                (Constant 1000)
                                Close 
                            )
                            (Pay
                                (Role "Bob")
                                (Party (Role "Alice"))
                                (Token "" "")
                                (Constant 1000)
                                Close 
                            )
                        )]
                    431686350279989 Close 
                )]
            216863500634957 Close 
        )]
    21686350063495 Close 
```

## Preliminaries

See [Lesson 0. Preliminaries](00-preliminaries.md) for information on setting up one's environment for using this tutorial.

The lesson assumes that the following environment variables have been set.
- `CARDANO_NODE_SOCKET_PATH`: location of Cardano node's socket.
- `CARDANO_TESTNET_MAGIC`: testnet magic number.

It also assumes that the Lender and Borrower parties have addresses, signing keys, and funds.
- Lender
    - [keys/alice.address](keys/alice.address): Cardano address for Alice
    - [keys/alice.skey](keys/alice.skey): location of signing key file for Alice
- Borrower
    - [keys/bob.address](keys/borrower.address): Cardano address for Bob
    - [keys/bob.skey](keys/borrower.skey): location of signing key file for Bob

### Access to Cardano node and Marlowe Runtime

If we're using [demeter.run](https://demeter.run/)'s Cardano Marlowe Runtime extension, then we already have access to Cardano Node and Marlowe Runtime. The followind commands will set the required environment variables to use a local docker deployment on the default ports. It will also set some supplementary environment variables.

In [2]:
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then

  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$(docker volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=2 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"

CARDANO_NODE_SOCKET_PATH = /ipc/node.socket
CARDANO_TESTNET_MAGIC = 2


In [65]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

NOW="$((`date -u +%s` * SECOND))"
echo NOW = "$NOW" POSIX milliseconds = "`date -d @$((NOW / SECOND))`"

NOW = 1686639290000 POSIX milliseconds = Tue Jun 13 06:54:50 AM UTC 2023


Note the test network magic number:
- `preprod` = 1
- `preview` = 2

### Alice's address and funds

Check that an address and key has been created for the lender. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [5]:
ALICE_SKEY=keys/alice.skey
ALICE_ADDR=$(cat keys/alice.address)
echo "ALICE_ADDR = $ALICE_ADDR"

ALICE_ADDR = addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq


Check that the lender has at least one hundred ada.

In [6]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8e63791b885c98d18ea3dc1034531f5b7f9f16b4708fa5dc15c2c4938d7a3bd7     1        87000000 lovelace + TxOutDatumNone
d7d3db5471e0ae38b60603613bf2a243dfa3857ab49807d1218fcdd33055af2d     0        917029304 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [7]:
echo "$EXPLORER_URL"/address/"$ALICE_ADDR"

https://preview.cardanoscan.io/address/addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq


### Bob's address and funds

Check that an address and key has been created for the borrower. If not, see "Creating Addresses and Signing Keys" in [Lesson 0. Preliminaries](00-preliminaries.md).

In [9]:
BOB_SKEY=keys/bob.skey
BOB_ADDR=$(cat keys/bob.address)
echo "BOB_ADDR = $BOB_ADDR"

BOB_ADDR = addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf


Check that the borrower has at least one hundred ada.

In [10]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BOB_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8e63791b885c98d18ea3dc1034531f5b7f9f16b4708fa5dc15c2c4938d7a3bd7     0        914430747 lovelace + TxOutDatumNone
d7d3db5471e0ae38b60603613bf2a243dfa3857ab49807d1218fcdd33055af2d     2        80000000 lovelace + TxOutDatumNone


One can view the address on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [11]:
echo "$EXPLORER_URL"/address/"$BOB_ADDR"

https://preview.cardanoscan.io/address/addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf


## Examine the contract

View the contract file as YAML.

In [12]:
json2yaml wolfram-contract.json

when:
- then:
    when:
    - then:
        when:
        - then:
            then:
              token:
                token_name: ''
                currency_symbol: ''
              to:
                party:
                  address: addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf
              then: close
              pay: 1000
              from_account:
                address: addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq
            if:
              value:
                value_of_choice:
                  choice_owner:
                    address: addr_test1vrxx3rjangevudlrejgp9m508uc26jd02n7lmdz365n4wrca35y5j
                  choice_name: WOLF_BTCUSD
              lt: 1000000
            else:
              token:
                token_name: ''
                currency_symbol: ''
              to:
                party:
                  address: addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq
              then: close
 

### \[Optional, but recommended\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe.iohk.io/).
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://https://play.marlowe.iohk.io//) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

See [Lesson 1](01-runtime-cli.ipynb) for an example of performing step 6.

## Transaction 1. Create the Contract

Marlowe CLI\'s command `marlowe-cli run initialize` will build the creation information for a Marlowe contract. We provide it the JSON files containing the contract and initial state. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

In [101]:
marlowe-cli run initialize \
  --permanently-without-staking \
  --contract-file wolfram-contract.json \
  --state-file wolfram-state.json \
  --out-file marlowe-1.json

We now use Marlowe CLI\'s `marlowe-cli run auto-execute` command to construct and submit the creation transaction.

In [102]:
TX_1=$(
marlowe-cli run auto-execute \
  --marlowe-out-file marlowe-1.json \
  --change-address "$BOB_ADDR" \
  --required-signer "$BOB_SKEY" \
  --out-file tx-1.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"


Fee: Lovelace 219445
Size: 1106 / 16384 = 6%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TX_1 = 06ab620ad60477c6c1c9c6658b57725012f470f3cb6181cfa191ad703c73c877


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [36]:
echo "$EXPLORER_URL"/transaction/"$TX_1?tab=utxo"

https://preview.cardanoscan.io/transaction/dcfbdc54d2f294cefddf5ddb8c81acf7c314cfba7e18759a9b49813db77b284d?tab=utxo


One can also examine the contract's UTxO using `cardano-cli`.

In [37]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_1#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dcfbdc54d2f294cefddf5ddb8c81acf7c314cfba7e18759a9b49813db77b284d     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "6c2f2ed776359af18012c9225795e4aff9b158f213cbce627667c8908244495f"


## Transaction 2. Alice deposit

Alice is going to bet some valuable cash she was saving for a rainy day

In [103]:
marlowe-cli run prepare \
  --deposit-account "$ALICE_ADDR" \
  --deposit-party "$ALICE_ADDR" \
  --deposit-amount "1000000" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file marlowe-1.json \
  --out-file marlowe-2.json

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1686641292000},POSIXTime {getPOSIXTime = 1686641652999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1686641292000},POSIXTime {getPOSIXTime = 1686641652999}), txInputs = [NormalInput (IDeposit "\"addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq\"" "\"addr_test1vrc79wasc0na5zglchugujuevu9ra9ehfjxwl4x9stgm9uc9dpguq\"" (Token "" "") 1000000)]}


Once again, use `marlowe-cli run auto-execute` to build and submit the transaction and then wait for confirmation.

In [104]:
TX_2=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_1#1" \
  --marlowe-in-file marlowe-1.json \
  --marlowe-out-file marlowe-2.json \
  --change-address "$ALICE_ADDR" \
  --required-signer "$ALICE_SKEY" \
  --out-file tx-2.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"


Fee: Lovelace 696180
Size: 2043 / 16384 = 12%
Execution units:
  Memory: 5608780 / 14000000 = 40%
  Steps: 1550506015 / 10000000000 = 15%


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [28]:
echo "$EXPLORER_URL"/transaction/"$TX_2?tab=utxo"

https://preview.cardanoscan.io/transaction/d7d3db5471e0ae38b60603613bf2a243dfa3857ab49807d1218fcdd33055af2d?tab=utxo


One can see that Alice has 1000 ada less than originally. 

In [74]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ALICE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5c5328866aa0cf7af63ffbb68b5c3a99a4cf993c56d682be8b62fcf97fcc07f9     0        75683649 lovelace + TxOutDatumNone
d7d3db5471e0ae38b60603613bf2a243dfa3857ab49807d1218fcdd33055af2d     0        917029304 lovelace + TxOutDatumNone


The Marlowe contract still has the 2 ada from its creation.

In [78]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


## Transaction 3. Bob deposit

Bob is unemployed because of his gampling problems. This bet is his only chance to find a good therapist.

In [79]:
marlowe-cli run prepare \
  --deposit-account "$BOB_ADDR" \
  --deposit-party "$BOB_ADDR" \
  --deposit-amount "1000000" \
  --invalid-before "$((`date -u +%s` * SECOND - 1 * MINUTE))" \
  --invalid-hereafter "$((`date -u +%s` * SECOND + 5 * MINUTE))" \
  --marlowe-file marlowe-2.json \
  --out-file marlowe-3.json

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1686639665000},POSIXTime {getPOSIXTime = 1686640025999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1686639665000},POSIXTime {getPOSIXTime = 1686640025999}), txInputs = [NormalInput (IDeposit "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\"" "\"addr_test1vrudhk63pclytll6vdt6hrda75al5a2jhtlkuqn0mh89ngsr929vf\"" (Token "" "") 1000000)]}


Once again, use `marlowe-cli run auto-execute` to build and submit the transaction and then wait for confirmation.

In [100]:
TX_3=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_1#0" \
  --marlowe-in-file marlowe-2.json \
  --marlowe-out-file marlowe-3.json \
  --change-address "$BOB_ADDR" \
  --required-signer "$BOB_SKEY" \
  --out-file tx-3.signed \
  --submit 600 \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

TxBodyScriptExecutionError [(ScriptWitnessIndexTxIn 1,ScriptErrorRedeemerPointsToUnknownScriptHash (ScriptWitnessIndexTxIn 1))]
TX_3 = 


One can view the transaction on a Cardano explorer. It sometimes takes thirty seconds or so for the transaction to be visible in an explorer.

In [34]:
echo "$EXPLORER_URL"/transaction/"$TX_3?tab=utxo"

https://preview.cardanoscan.io/transaction/8e63791b885c98d18ea3dc1034531f5b7f9f16b4708fa5dc15c2c4938d7a3bd7?tab=utxo


Now we wait for Oracle to decide the winner

In [90]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ALICE_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5c5328866aa0cf7af63ffbb68b5c3a99a4cf993c56d682be8b62fcf97fcc07f9     0        75683649 lovelace + TxOutDatumNone
d7d3db5471e0ae38b60603613bf2a243dfa3857ab49807d1218fcdd33055af2d     0        917029304 lovelace + TxOutDatumNone


The borrower now has about 5 ada (the loan's interest) less than originally.

In [91]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BOB_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
77cbdf3d315ca6f6ab33adf708a511b88e269e53227f45d754d9f284833e4d84     0        78306842 lovelace + TxOutDatumNone
dcfbdc54d2f294cefddf5ddb8c81acf7c314cfba7e18759a9b49813db77b284d     0        907632079 lovelace + TxOutDatumNone


The Marlowe contract has closed, so there is no output to its script address.